In [59]:
import pandas as pd
import dask as dd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.formula.api as smf
import statistics

from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats

In [91]:
def basePrep(df):
	df['rndf'] = df['rndf'].map({"Yes": 1, "No": 0})
	df['pdynamic'] = df['pdynamic'].map({'new_entrant': 1, 'incumbent': 0})	
	df["SNATION_PARTAL".lower()] = df["SNATION_PARTAL".lower()].map({"Y": 1, "N": 0})
	df['cr_bor_part'] = df['cr_bor_part'].map({"Y": 1, "N": 0})
	df['mfgf'] = df['mfgf'].map({"Yes": 1, "No": 0})
	df['jvf'] = df['jvf'].map({'Yes': 1, 'No': 0})
	
	cols = ['rndf', 'jvf','ddist_bin','ddist_int', 'ddist_abs', 'avg_emp_pp', 'public_count', 'pdynamic', "SNATION_PARTAL".lower(), 'cr_bor_part', 'mfgf']
	df = df[cols].dropna()
	
	return df

def variation(df):
	cols = ['rndf', 'ddist_bin','ddist_int', 'ddist_abs', 'avg_emp_pp', 'public_count', 
			"SNATION_PARTAL".lower(), 'cr_bor_part', 'mfgf']
	X = (df[cols]).astype(float)
	y = df['pdynamic'].astype(float)
	X_constant = sm.add_constant(X, prepend=False)

	logit_model = GLM(y, X_constant, family=families.Binomial())
	logit_results = logit_model.fit()
	print(logit_results.summary())

## Try all potential variables

In [75]:
df = pd.read_csv('./data/JV_data_dist.csv')
df = basePrep(df)

variation(df)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  729
Model:                            GLM   Df Residuals:                      719
Model Family:                Binomial   Df Model:                            9
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -429.40
Date:                Wed, 15 Jun 2022   Deviance:                       858.80
Time:                        21:07:40   Pearson chi2:                     765.
No. Iterations:                    19   Pseudo R-squ. (CS):             0.1283
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf               0.2649      0.312      0.

## Filter only R&D alliances

In [76]:
df2 = pd.read_csv('./data/JV_data_dist.csv')

df2 = df2[df2['rndf'] == "Yes"]
df2 = basePrep(df2)

variation(df2)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                   66
Model:                            GLM   Df Residuals:                       58
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -33.248
Date:                Wed, 15 Jun 2022   Deviance:                       66.496
Time:                        21:07:56   Pearson chi2:                     62.4
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1755
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf               3.5145      1.679      2.

## Filter for alliance SIC

In [78]:
df3 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df3 = df3[df3['sic'].str.contains('3711') == True]
df3 = basePrep(df3)

variation(df3)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  311
Model:                            GLM   Df Residuals:                      302
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -186.90
Date:                Wed, 15 Jun 2022   Deviance:                       373.80
Time:                        21:08:48   Pearson chi2:                     330.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1600
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.5675      0.725     -0.

## Filter for same nation participants and alliance

In [82]:
df4 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df4 = df4[df4['snation_partal'] == "Y"]
df4 = basePrep(df4)

variation(df4)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  154
Model:                            GLM   Df Residuals:                      146
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -74.552
Date:                Wed, 15 Jun 2022   Deviance:                       149.10
Time:                        21:09:34   Pearson chi2:                     156.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.06850
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.5855      0.659     -0.

## Filter for Manufacturing agreement flags

In [84]:
df5 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df5 = df5[df5['mfgf'] == "Yes"]
df5 = basePrep(df5)

variation(df5)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  481
Model:                            GLM   Df Residuals:                      473
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -295.39
Date:                Wed, 15 Jun 2022   Deviance:                       590.77
Time:                        21:10:01   Pearson chi2:                     514.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1203
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf               0.5440      0.486      1.

In [86]:
df6 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df6 = df6[df6['mfgf'] == "Yes"]
df6 = df6[df6['sic'].str.contains('3711') == True]
df6 = basePrep(df6)

variation(df6)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  267
Model:                            GLM   Df Residuals:                      259
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -161.63
Date:                Wed, 15 Jun 2022   Deviance:                       323.27
Time:                        21:10:57   Pearson chi2:                     281.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1524
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.3928      0.877     -0.

In [89]:
df7 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df7 = df7[df7['mfgf'] == "Yes"]
df7 = df7[df7['sic'].str.contains('3711') == True]
df7 = df7[df7['public_count'] >= 1]
df7 = df7[df7['nump'] == 2]
df7 = basePrep(df7)

variation(df7)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  202
Model:                            GLM   Df Residuals:                      194
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -115.46
Date:                Wed, 15 Jun 2022   Deviance:                       230.93
Time:                        21:23:28   Pearson chi2:                     199.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.2005
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.5084      1.332     -0.

## Test for final H1

In [90]:
dfH = pd.read_csv("./data/JV_data_dist.csv")

dfH = dfH[dfH['mfgf'] == "Yes"]
dfH = dfH[dfH['sic'].str.contains('3711') == True]
dfH = dfH[dfH['public_count'] >= 1]
dfH = dfH[dfH['nump'] == 2]
dfH = basePrep(dfH)

cols = ['ddist_int']
X = (dfH[cols]).astype(float)
y = dfH['pdynamic'].astype(float)
X_constant = sm.add_constant(X, prepend=False)

logit_model = GLM(y, X_constant, family=families.Binomial())
logit_results = logit_model.fit()
print(logit_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  202
Model:                            GLM   Df Residuals:                      200
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -130.99
Date:                Wed, 15 Jun 2022   Deviance:                       261.97
Time:                        21:28:24   Pearson chi2:                     202.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.06772
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ddist_int      0.0004      0.000      3.632      0.0

In [99]:
dfH2 = pd.read_csv("./data/JV_data_dist.csv")

dfH2 = dfH2[dfH2['mfgf'] == "Yes"]
dfH2 = dfH2[dfH2['sic'].str.contains('3711') == True]
dfH2 = dfH2[dfH2['public_count'] >= 1]
dfH2 = dfH2[dfH2['nump'] == 2]
dfH2 = basePrep(dfH2)

cols = ['ddist_int', 'public_count', 'rndf', 'snation_partal', 'avg_emp_pp']
X = (dfH2[cols]).astype(float)
y = dfH2['jvf'].astype(float)
X_constant = sm.add_constant(X, prepend=False)

logit_model = GLM(y, X_constant, family=families.Binomial())
logit_results = logit_model.fit()
print(logit_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    jvf   No. Observations:                  202
Model:                            GLM   Df Residuals:                      196
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -108.12
Date:                Wed, 15 Jun 2022   Deviance:                       216.24
Time:                        22:57:51   Pearson chi2:                     202.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1043
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
ddist_int          0.0005      0.000      3.